In [0]:
files = dbutils.fs.ls('mnt/demo-datasets/bookstore') # finding all the files in the directory 

In [0]:
display(files)

path,name,size,modificationTime
dbfs:/mnt/demo-datasets/bookstore/books-cdc/,books-cdc/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-csv/,books-csv/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-csv-new/,books-csv-new/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-streaming/,books-streaming/,0,0
dbfs:/mnt/demo-datasets/bookstore/customers-json/,customers-json/,0,0
dbfs:/mnt/demo-datasets/bookstore/customers-json-new/,customers-json-new/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders/,orders/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-json-raw/,orders-json-raw/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-json-streaming/,orders-json-streaming/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-new/,orders-new/,0,0


In [0]:
%sql
CREATE TABLE books
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS (
  header = "true",
  delimiter = ";",
  path = 'dbfs:/mnt/demo-datasets/bookstore/books-csv/export_*.csv'
);
-- LOCATION 'dbfs:/mnt/demo-datasets/bookstore/books/'

--dbfs:/mnt/demo-datasets/bookstore/books-csv/export_001.csv

In [0]:
%sql
SELECT * FROM books;

book_id,title,author,category,price
B07,The Hundred-Page Machine Learning,Andriy Burkov,Computer Science,33.0
B08,Quantum Computing for Everyone,Chris Bernhardt,Computer Science,41.0
B09,Advanced Data Structures,Peter Brass,Computer Science,24.0
B10,Beginning Database Design Solutions,Rod Stephens,Computer Science,44.0
B11,Business Intelligence for Dummies,Swain Scheps,Computer Science,38.0
B12,Big Data in Practice,Bernard Marr,Computer Science,30.0
B01,The Soul of a New Machine,Tracy Kidder,Computer Science,49.0
B02,Learning JavaScript Design Patterns,Addy Osmani,Computer Science,28.0
B03,Make Your Own Neural Network,Tariq Rashid,Computer Science,35.0
B04,Robot Dynamics and Control,Mark W. Spong,Computer Science,20.0


READING DELTA TABLE AS STREAM OF DATA

In [0]:
(spark.readStream
 .table("books")
 .createOrReplaceTempView("books_streaming_temp_vw"))

Querying the temp view 

In [0]:
%sql
SELECT * FROM books_streaming_temp_vw;

book_id,title,author,category,price
B07,The Hundred-Page Machine Learning,Andriy Burkov,Computer Science,33.0
B08,Quantum Computing for Everyone,Chris Bernhardt,Computer Science,41.0
B09,Advanced Data Structures,Peter Brass,Computer Science,24.0
B10,Beginning Database Design Solutions,Rod Stephens,Computer Science,44.0
B11,Business Intelligence for Dummies,Swain Scheps,Computer Science,38.0
B12,Big Data in Practice,Bernard Marr,Computer Science,30.0
B01,The Soul of a New Machine,Tracy Kidder,Computer Science,49.0
B02,Learning JavaScript Design Patterns,Addy Osmani,Computer Science,28.0
B03,Make Your Own Neural Network,Tariq Rashid,Computer Science,35.0
B04,Robot Dynamics and Control,Mark W. Spong,Computer Science,20.0


Some Aggregation on streaming temp view

In [0]:
%sql
SELECT author, COUNT(book_id) total_books FROM books_streaming_temp_vw 
GROUP BY author;

author,total_books
Mark W. Spong,1
Chris Bernhardt,1
Tariq Rashid,1
Peter Brass,1
Luciano Ramalho,1
Addy Osmani,1
Andriy Burkov,1
Tracy Kidder,1
Swain Scheps,1
François Chollet,1


In [0]:
%sql
SELECT * FROM books_streaming_temp_vw 
ORDER BY author; -- sorting is not supported by streaming temp view

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-693258673127663>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-693258673127663>:5, in ____databricks_percent_sql()
      3 import base64
      4 df = spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSBib29rc19zdHJlYW1pbmdfdGVtcF92dyAKT1JERVIgQlkgYXV0aG9y").decode())
----> 5 display(df)
      6 return df

File /databricks/python_shell/dbruntime/display.py:76, in Display.display(self, input, *args, **kwargs)
     74 elif isinstance(input, DataFrame):
     75     if hasattr(input, "isStreaming") and input.isStreaming:
---> 76         handleStreamingDataFrame(input, self.sc, self.sqlContext, self.entry_point, kwargs)
     77     else:
     78         if kwargs.get('streamName'):

File /databricks/pyth

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW author_count_temp_view -- author_count_temp_view is also streaming temporary view
AS SELECT author,COUNT(book_id) total_books FROM books_streaming_temp_vw 
GROUP BY author;

In [0]:
(spark.table("author_count_temp_view")                               
      .writeStream  
      .trigger(processingTime='4 seconds')
      .outputMode("complete") 
      .option("checkpointLocation", "dbfs:/mnt/demo-datasets/author_counts_checkpoint")
      .table("author_counts")
)


Out[120]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fc1671dccd0>

In [0]:
%sql
SELECT * FROM author_counts; 
-- Note: This is not a streaming query

author,total_books
François Chollet,1
Chris Bernhardt,1
Luciano Ramalho,1
Mark W. Spong,1
Andriy Burkov,1
Tariq Rashid,1
Tracy Kidder,1
Swain Scheps,1
Rod Stephens,1
Bernard Marr,1


In [0]:
spark.sql("SET spark.sql.hive.convertMetastoreParquet=false")

Out[131]: DataFrame[key: string, value: string]

In [0]:
%sql
INSERT INTO books VALUES ("B16", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25);

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-693258673127670>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-693258673127670>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("SU5TRVJUIElOVE8gYm9va3MgVkFMVUVTICgiQjE2IiwgIkhhbmRzLU9uIERlZXAgTGVhcm5pbmcgQWxnb3JpdGhtcyB3aXRoIFB5dGhvbiIsICJTdWRoYXJzYW4gUmF2aWNoYW5kaXJhbiIsICJDb21wdXRlciBTY2llbmNlIiwgMjUp").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     5

In [0]:
%sql
-- The graph of streaming query keeps changing as the new data arrives 
-- lets insert new data in our source table(books)

INSERT INTO books
 values ("B16", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25),
        ("B17", "Neural Network Methods in Natural Language Processing", "Yoav Goldberg", "Computer Science", 30),
        ("B18", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35);

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-693258673127667>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-693258673127667>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("LS0gVGhlIGdyYXBoIG9mIHN0cmVhbWluZyBxdWVyeSBrZWVwcyBjaGFuZ2luZyBhcyB0aGUgbmV3IGRhdGEgYXJyaXZlcyAKLS0gbGV0cyBpbnNlcnQgbmV3IGRhdGEgaW4gb3VyIHNvdXJjZSB0YWJsZShib29rcykKCklOU0VSVCBJTlRPIGJvb2tzIFZBTFVFUyAoCigiQjE5IiwgIkludHJvZHVjdGlvbiB0byBNb2RlbGluZyBhbmQgU2ltdWxhdGlvbiIsICJNYXJrIFcuIFNwb25nIiwgIkNvbXB1dGVyIFNjaWVuY2UiLCAyNSksCigiQjIwIiwgIlJvYm90IE1vZGVsaW5nIGFuZCBDb250cm9sIiwgIk1hcmsgVy4gU3BvbmciLCAiQ29tcHV0ZXIgU2NpZW5jZSIsIDMwKSwKKCJCMjEiLCAiVHVyaW5nJ3MgVmlzaW9uOiBUaGUgQmlydG